In [2]:
import numpy as np
import random

# Helper Functions
def calculate_fitness(chromosome, n, t):
    penalties_overlap = 0
    penalties_consistency = 0

    # Reshape chromosome into matrix format for easier processing
    schedule = np.array(chromosome).reshape((t, n))

    # Overlap penalty
    for timeslot in schedule:
        penalties_overlap += max(0, sum(timeslot) - 1)

    # Consistency penalty
    for course in range(n):
        appearances = sum(schedule[:, course])
        penalties_consistency += abs(appearances - 1)

    # Fitness Calculation
    fitness = -(penalties_overlap + penalties_consistency)
    return fitness

# Generate random initial population
def generate_population(pop_size, n, t):
    population = []
    for _ in range(pop_size):
        chromosome = [random.randint(0, 1) for _ in range(n * t)]
        if sum(chromosome) == 0:  # Avoid chromosomes with all zeros
            chromosome[random.randint(0, n * t - 1)] = 1
        population.append(chromosome)
    return population

# Single-point crossover
def single_point_crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

# Two-point crossover
def two_point_crossover(parent1, parent2):
    point1 = random.randint(0, len(parent1) - 2)
    point2 = random.randint(point1 + 1, len(parent1) - 1)
    child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]
    return child1, child2

# Mutation
def mutate(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]  # Flip bit
    return chromosome

# Genetic Algorithm
def genetic_algorithm(n, t, courses, pop_size=10, max_generations=100, mutation_rate=0.01):
    population = generate_population(pop_size, n, t)
    best_fitness = float('-inf')
    best_chromosome = None

    for generation in range(max_generations):
        fitness_scores = [calculate_fitness(chromosome, n, t) for chromosome in population]

        # Ensure fitness scores are usable as weights
        total_fitness = sum(fitness_scores)
        if total_fitness == 0:
            # Fallback: Select parents randomly if all fitness scores are zero
            selected = random.choices(population, k=pop_size // 2)
        else:
            # Normalize weights to create a probability distribution
            normalized_weights = [score / total_fitness for score in fitness_scores]
            selected = random.choices(population, weights=normalized_weights, k=pop_size // 2)

        # Select the best chromosome
        for i, fitness in enumerate(fitness_scores):
            if fitness > best_fitness:
                best_fitness = fitness
                best_chromosome = population[i]

        # Crossover
        offspring = []
        for i in range(0, len(selected), 2):
            if i + 1 < len(selected):
                child1, child2 = single_point_crossover(selected[i], selected[i + 1])
                offspring.extend([child1, child2])

        # Mutation
        offspring = [mutate(child, mutation_rate) for child in offspring]

        # Create new population
        population = offspring

    return best_chromosome, best_fitness

# Example Input
n = 3  # Number of courses
t = 3  # Number of timeslots
courses = ["CSE110", "MAT110", "PHY112"]

# Run the Genetic Algorithm
best_chromosome, best_fitness = genetic_algorithm(n, t, courses)

# Output
print("Best Chromosome:", ''.join(map(str, best_chromosome)))
print("Fitness Value:", best_fitness)

# Two-point Crossover Example
parent1 = [1, 1, 0, 1, 1, 0, 0, 1, 0]
parent2 = [0, 0, 1, 0, 0, 1, 1, 0, 1]
child1, child2 = two_point_crossover(parent1, parent2)
print("Parent 1:", parent1)
print("Parent 2:", parent2)
print("Child 1:", child1)
print("Child 2:", child2)


Best Chromosome: 010100001
Fitness Value: 0
Parent 1: [1, 1, 0, 1, 1, 0, 0, 1, 0]
Parent 2: [0, 0, 1, 0, 0, 1, 1, 0, 1]
Child 1: [0, 0, 1, 0, 0, 1, 0, 1, 0]
Child 2: [1, 1, 0, 1, 1, 0, 1, 0, 1]
